<a href="https://colab.research.google.com/github/Shaunofthedead19/DEZoomcamp2024/blob/main/cohorts/2024/workshops/dlt_resources/homework_starter.ipynbhomework_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [9]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

In [16]:
#Imports
import dlt
import duckdb

Solution 1

In [6]:
sum = 0
limit = 5
generator = square_root_generator(limit)

for n in range(limit):
  sum+=next(generator)

#Question 1: Sum of outputs
print("Sum of values:", sum)

Sum of values: 8.382332347441762


Solution 2

In [8]:
limit=13
generator_2=square_root_generator(limit)

n=0
for value in generator_2:
  n+=1
  if n < 13:
    continue
  else:
    print("13th value is: ", value)

13th value is:  3.605551275463989


Solution 3

In [13]:
#Loading pipeline_1 to duckdb
p1_pipeline = dlt.pipeline(destination = 'duckdb', dataset_name='people')

#Running p1_pipeline
info = p1_pipeline.run(people_1(),
                       table_name = "people",
                       write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708250481.3772933 is LOADED and contains no failed jobs


In [15]:
#Loading Pipeline_2 to duckdb
p2_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

#Running p2_pipeline
info = p2_pipeline.run(people_2(),
                       table_name="people",
                       write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708250683.1801634 is LOADED and contains no failed jobs


In [17]:
#connecting to duckdb DB and displaying available tables
connection = duckdb.connect(f"{p1_pipeline.pipeline_name}.duckdb")

connection.sql(f"SET search_path = '{p1_pipeline.dataset_name}'")
display(connection.sql("show tables"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [18]:
#Checking data in people table
display(connection.sql("SELECT * from people"))

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708250481.3772933 │ Tcn78vRXjmGlMQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708250481.3772933 │ WHLS1iW6ijw6YQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708250481.3772933 │ ASm5OpVNUaOMzQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708250481.3772933 │ vG2JDE/cKLhtxQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708250481.3772933 │ gbLhJ340gGnirA │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708250683.1801634 │ xlr5QDDolPumCA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708250683.1801634 │ 5gH7BoNnTQVqUw │ Job_4      │
│     5 │ 

In [23]:
print("Sum of all ages in people table" ,connection.execute("SELECT SUM(AGE) FROM PEOPLE").fetchall()[0][0])

Sum of all ages in people table 353


Solution 4

In [24]:
#Loading pipeline to duckdb
generators = dlt.pipeline(destination = 'duckdb', dataset_name='people_merged')

#Running pipeline 'generators'
info = generators.run(people_1(),
                       table_name = "people_merged",
                       write_disposition="replace",
                      primary_key="ID")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708252122.8733745 is LOADED and contains no failed jobs


In [25]:
#Running 'generators' with people_2
info = generators.run(people_2(),
                       table_name = "people_merged",
                       write_disposition="merge",
                      primary_key="ID")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708252249.7254548 is LOADED and contains no failed jobs


In [26]:
#Connecting to duckdb DB and showing tables
connection = duckdb.connect(f"{generators.pipeline_name}.duckdb")

connection.sql(f"SET search_path = '{generators.dataset_name}'")
display(connection.sql("show tables"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merged       │
└─────────────────────┘

In [28]:
#Displaying contents of people_merged
display(connection.sql("SELECT * FROM people_merged"))

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708252122.8733745 │ C13jHBnCDX/yeQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708252122.8733745 │ kPu9NibQR66TjQ │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1708252249.7254548 │ qc0nzQ58Nzy5KQ │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1708252249.7254548 │ mPi82txyCHvlRw │ Job_7      │
│     8 │ Person_8 │    38 │ City_B  │ 1708252249.7254548 │ 69AKJVDRM56ZuQ │ Job_8      │
│     4 │ Person_4 │    34 │ City_B  │ 1708252249.7254548 │ 29QiXzu/hBWelw │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1708252249.7254548 │ vpeNxIl1VNGHnw │ Job_3      │
│     6 │ 

In [30]:
print("Sum of all ages in people table" ,connection.execute("SELECT SUM(AGE) FROM PEOPLE_MERGED").fetchall()[0][0])

Sum of all ages in people table 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX